In [1]:
!nvidia-smi

Tue Jan  3 07:44:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install transformers

     |████████████████████████████████| 4.9 MB 8.7 MB/s 
     |████████████████████████████████| 498.0 MB 13 kB/s 
     |████████████████████████████████| 1.4 MB 66.6 MB/s 
     |████████████████████████████████| 5.8 MB 77.0 MB/s 
     |████████████████████████████████| 462 kB 76.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 31.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.0 MB/s 
     |████████████████████████████████| 182 kB 82.3 MB/s 


In [3]:
!pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 34.0 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
     |████████████████████████████████| 118 kB 72.5 MB/s 
     |████████████████████████████████| 238 kB 76.6 MB/s 
     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 352 kB 56.8 MB/s 
     |████████████████████████████████| 1.3 MB 47.7 MB/s 


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 30.2 MB/s 
     |████████████████████████████████| 213 kB 81.3 MB/s 
     |████████████████████████████████| 132 kB 78.0 MB/s 
     |████████████████████████████████| 127 kB 71.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
import pandas as pd
import datetime
import numpy as np
import random
from datasets import Dataset

tf.get_logger().setLevel('ERROR')

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [6]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [7]:
# Read CSV data in
file_path = '/content/drive/MyDrive/labelled_marketaux_news_combined_2022-10-15.csv'

df = pd.read_csv(file_path)
df.head()

,Unnamed: 0,Unnamed: 0.1,Stock,Title,Date/Time Published,Source,Url,Relevant Texts,Sentiment Score,Match Score,key,3m,6m,1y
0,0,0,A,Biotech : comment investir sur les biotechnolo...,2021-04-22,trading-attitude.com,https://www.trading-attitude.com/biotech-comme...,Agilent Technologies... Agilent Technologies I...,NaN,26.909225,A2021-04-22,0.0,0.0,0.0
1,1,43,A,Agilent Technologies : LC/MS Instruments Recei...,2021-04-22,marketscreener.com,https://www.marketscreener.com/quote/stock/AGI...,Reinforcing Agilent’s commitment to sustainabl...,0.553500,66.971190,A2021-04-22,0.0,0.0,0.0
2,2,44,A,Clinical Oncology Next-generation Sequencing M...,2021-04-22,globalbankingandfinance.com,https://www.globalbankingandfinance.com/clinic...,"., PerkinElmer Inc.and Agilent Technologies, I...",0.526700,29.795845,A2021-04-22,0.0,0.0,0.0
3,3,1,A,Worldwide Endocrine Testing Industry To 2026 -...,2021-12-02,thestreet.com,https://www.thestreet.com/press-releases/world...,Competitive Landscape:... The report has also ...,0.272233,35.761560,A2021-12-02,1.0,0.0,NaN
4,4,57,A,InMode Ltd. : De retour sur un niveau important,2021-12-02,zonebourse.com,https://www.zonebourse.com/cours/action/INMODE...,. -1.26% 55 902 BOSTON SCIENTIFIC CORPORATI.. ...,NaN,25.916040,A2021-12-02,1.0,0.0,NaN


In [8]:
# convert date from string to date objects
df['Date/Time Published'] = pd.to_datetime(df['Date/Time Published'])
df.head()

,Unnamed: 0,Unnamed: 0.1,Stock,Title,Date/Time Published,Source,Url,Relevant Texts,Sentiment Score,Match Score,key,3m,6m,1y
0,0,0,A,Biotech : comment investir sur les biotechnolo...,2021-04-22,trading-attitude.com,https://www.trading-attitude.com/biotech-comme...,Agilent Technologies... Agilent Technologies I...,NaN,26.909225,A2021-04-22,0.0,0.0,0.0
1,1,43,A,Agilent Technologies : LC/MS Instruments Recei...,2021-04-22,marketscreener.com,https://www.marketscreener.com/quote/stock/AGI...,Reinforcing Agilent’s commitment to sustainabl...,0.553500,66.971190,A2021-04-22,0.0,0.0,0.0
2,2,44,A,Clinical Oncology Next-generation Sequencing M...,2021-04-22,globalbankingandfinance.com,https://www.globalbankingandfinance.com/clinic...,"., PerkinElmer Inc.and Agilent Technologies, I...",0.526700,29.795845,A2021-04-22,0.0,0.0,0.0
3,3,1,A,Worldwide Endocrine Testing Industry To 2026 -...,2021-12-02,thestreet.com,https://www.thestreet.com/press-releases/world...,Competitive Landscape:... The report has also ...,0.272233,35.761560,A2021-12-02,1.0,0.0,NaN
4,4,57,A,InMode Ltd. : De retour sur un niveau important,2021-12-02,zonebourse.com,https://www.zonebourse.com/cours/action/INMODE...,. -1.26% 55 902 BOSTON SCIENTIFIC CORPORATI.. ...,NaN,25.916040,A2021-12-02,1.0,0.0,NaN


In [9]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

# Convert all titles to lowercase
df['Relevant Texts'] = df['Relevant Texts'].apply(lambda x: x.lower())
df.head()
# Remove entries where 3m is NaN
df = df.loc[~df['3m'].isnull()]
# df = df.loc[~df['6m'].isnull()]
# df = df.loc[~df['1y'].isnull()]

# keep titles with ascii only
df = df[df.apply(lambda r: is_ascii(r['Relevant Texts']), axis=1)]

print(df.head())
print(len(df.index))

   Unnamed: 0  Unnamed: 0.1 Stock  \
2           2            44     A   
3           3             1     A   
7           7             4     A   
8           8             5     A   
9           9            28     A   

                                               Title Date/Time Published  \
2  Clinical Oncology Next-generation Sequencing M...          2021-04-22   
3  Worldwide Endocrine Testing Industry To 2026 -...          2021-12-02   
7                         A Look Into Agilent's Debt          2021-11-18   
8  Worldwide Polymerase Chain Reaction Industry T...          2021-11-18   
9  Global Metabolomics Technologies and Markets, ...          2021-11-18   

                        Source  \
2  globalbankingandfinance.com   
3                thestreet.com   
7                 benzinga.com   
8                thestreet.com   
9               prnewswire.com   

                                                 Url  \
2  https://www.globalbankingandfinance.com/clinic...   
3  

In [10]:
# check the number of tickers and rank them by frequency of texts
df.groupby('Stock')['Relevant Texts'].nunique()

Stock
A       72
AA      49
AAL     62
AAON    37
AAP     56
        ..
SYY     44
T       60
TAC     59
TAK     55
TAL     55
Name: Relevant Texts, Length: 1199, dtype: int64

In [11]:
df.loc[df['Stock'] == 'A']

,Unnamed: 0,Unnamed: 0.1,Stock,Title,Date/Time Published,Source,Url,Relevant Texts,Sentiment Score,Match Score,key,3m,6m,1y
2,2,44,A,Clinical Oncology Next-generation Sequencing M...,2021-04-22,globalbankingandfinance.com,https://www.globalbankingandfinance.com/clinic...,"., perkinelmer inc.and agilent technologies, i...",0.526700,29.795845,A2021-04-22,0.0,0.0,0.0
3,3,1,A,Worldwide Endocrine Testing Industry To 2026 -...,2021-12-02,thestreet.com,https://www.thestreet.com/press-releases/world...,competitive landscape:... the report has also ...,0.272233,35.761560,A2021-12-02,1.0,0.0,NaN
7,7,4,A,A Look Into Agilent's Debt,2021-11-18,benzinga.com,https://www.benzinga.com/news/21/11/24177458/a...,"over the past three months, shares of agilent ...",-0.361200,36.274900,A2021-11-18,1.0,1.0,NaN
8,8,5,A,Worldwide Polymerase Chain Reaction Industry T...,2021-11-18,thestreet.com,https://www.thestreet.com/press-releases/world...,competitive landscape... the report has also a...,0.272233,36.964996,A2021-11-18,1.0,1.0,NaN
9,9,28,A,"Global Metabolomics Technologies and Markets, ...",2021-11-18,prnewswire.com,https://www.prnewswire.com:443/news-releases/g...,approaches for metabolomic technology and comp...,0.236040,54.815590,A2021-11-18,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,97,96,A,Infertility Testing and Treatment Market: A ri...,2021-08-12,news.google.com,https://www.biospace.com/article/infertility-t...,"agilent technologies, inc.... f. hoffmann-la r...",-0.226300,26.258660,A2021-08-12,0.0,1.0,0.0
98,98,97,A,Genomics Market to Register USD 21.10 bn growt...,2021-08-12,prnewswire.com,https://www.prnewswire.com:443/news-releases/g...,"customer behavior... optimistic, probable, and...",0.363450,28.734703,A2021-08-12,0.0,1.0,0.0
99,99,101,A,Should You Buy Agilent Technologies (A) Ahead ...,2021-08-12,zacks.com,https://www.zacks.com/stock/news/1780999/shoul...,should you buy agilent technologies (a) ahead ...,0.000000,29.324642,A2021-08-12,0.0,1.0,0.0
105,105,114,A,Stocks That Hit 52-Week Highs On Thursday,2021-07-29,benzinga.com,https://www.benzinga.com/news/21/07/22224542/s...,agilent technologies (nyse:a) shares achieved ...,0.680800,7.059691,A2021-07-29,0.0,1.0,0.0


In [12]:
# We want to split train:test by date
min_date = df['Date/Time Published'].min()
max_date = df['Date/Time Published'].max()
print('Min Date:', min_date)
print('Max Date:', max_date)

max_date - min_date

Min Date: 2021-01-04 00:00:00
Max Date: 2022-05-19 00:00:00


Timedelta('500 days 00:00:00')

In [13]:
train_delta = datetime.timedelta(days=300)
val_delta = datetime.timedelta(days=45)
train_cut_off_date = min_date + train_delta
val_cut_off_date = train_cut_off_date + val_delta
print(train_cut_off_date)
print(val_cut_off_date)

2021-10-31 00:00:00
2021-12-15 00:00:00


In [14]:
# split data by cut off date
# train:val:test - 0.75: 0.10: 0.15
df_train = df.loc[df['Date/Time Published'] <= train_cut_off_date]
df_val = df.loc[(df['Date/Time Published'] > train_cut_off_date) & (df['Date/Time Published'] <= val_cut_off_date)]
df_test = df.loc[df['Date/Time Published'] > val_cut_off_date]
print(len(df_train.index))
print(len(df_val.index))
print(len(df_test.index))
total = len(df_train.index) + len(df_test.index) + len(df_val.index)
print('Ratio of train to test -> ', len(df_train.index)/ total, len(df_val.index)/ total, len(df_test.index)/total)

29406
4290
5522
Ratio of train to test ->  0.7498087612830843 0.10938854607578152 0.14080269264113418


In [15]:
df_test

,Unnamed: 0,Unnamed: 0.1,Stock,Title,Date/Time Published,Source,Url,Relevant Texts,Sentiment Score,Match Score,key,3m,6m,1y
12,12,9,A,"With 22.2% CAGR, Next-Generation Sequencing Ma...",2022-05-05,globenewswire.com,https://www.globenewswire.com/news-release/202...,"hoffmann-la roche ltd... agilent technologies,...",0.000000,17.656471,A2022-05-05,2.0,NaN,NaN
13,13,41,A,U.S. Accounts for Over 37% Revenue Share of th...,2022-05-05,globenewswire.com,https://www.globenewswire.com/news-release/202...,flag=rc&rep_id=7306... key players in the micr...,0.000000,23.182043,A2022-05-05,2.0,NaN,NaN
22,22,13,A,Arbovirus Testing Market: 5.00% Y-O-Y Growth R...,2022-04-07,prnewswire.com,https://www.prnewswire.com:443/news-releases/a...,structure fragmented yoy growth (%) 5.0 region...,0.098825,41.700190,A2022-04-07,0.0,NaN,NaN
24,24,14,A,Agilent to Participate at Cowen and Barclays H...,2022-02-24,marketscreener.com,https://www.marketscreener.com/quote/stock/AGI...,agilent technologies inc. (nyse: a) today anno...,0.566200,74.454500,A2022-02-24,0.0,0.0,NaN
25,25,15,A,10 Stocks in Focus After Posting Their Earning...,2022-02-24,insidermonkey.com,https://www.insidermonkey.com/blog/10-stocks-i...,"(nyse:tdoc) and agilent technologies, inc. (ny...",0.612400,34.842155,A2022-02-24,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57886,57886,61976,TAL,"EdTech Firm Fires 60,000 in Worst Cuts Since C...",2022-01-10,techinvestornews.com,https://www.techinvestornews.com/Tech-News/Lat...,the three biggest operators in the space -- in...,0.000000,28.195990,TAL2022-01-10,0.0,2.0,NaN
57892,57892,61951,TAL,3 Top Penny Stocks to Add to Your Watchlist in...,2022-01-12,pennystocks.com,https://pennystocks.com/featured/2022/01/12/3-...,tal education group (nyse: tal)... with a 5% g...,0.814000,25.599716,TAL2022-01-12,1.0,2.0,NaN
57907,57907,61966,TAL,44 Stocks Moving In Thursday's Mid-Day Session,2021-12-30,markets.businessinsider.com,https://markets.businessinsider.com/news/stock...,tal education group (nyse:tal) gained 14.5% to...,0.636900,27.168960,TAL2021-12-30,1.0,2.0,NaN
57908,57908,61975,TAL,"Why TAL Education, New Oriental Education And ...",2021-12-30,markets.businessinsider.com,https://markets.businessinsider.com/news/stock...,"shares of several chinese stocks, including ta...",0.248367,86.994545,TAL2021-12-30,1.0,2.0,NaN


In [16]:
df_train['3m'].value_counts()

0.0    12162
1.0    10239
2.0     7005
Name: 3m, dtype: int64

In [17]:
df_test['3m'].value_counts()

0.0    2069
1.0    1772
2.0    1681
Name: 3m, dtype: int64

In [18]:
df_val['3m'].value_counts()

0.0    1646
1.0    1380
2.0    1264
Name: 3m, dtype: int64

In [19]:
# check if train and test splits are balanced
df_train.groupby('Stock')['Relevant Texts'].nunique()

Stock
A       38
AA      26
AAL     35
AAON    22
AAP     33
        ..
SYY     32
T       40
TAC     40
TAK     40
TAL     38
Name: Relevant Texts, Length: 1198, dtype: int64

In [20]:
df_test.groupby('Stock')['Relevant Texts'].nunique()

Stock
A       26
AA      17
AAL     22
AAON    14
AAP     18
        ..
SYY      6
T       11
TAC      9
TAK      9
TAL     12
Name: Relevant Texts, Length: 826, dtype: int64

In [21]:
num_classes = df_train['3m'].nunique()
print(num_classes)

3


In [22]:
df_train_filtered = df_train.filter(items=['Relevant Texts', '3m'])
df_val_filtered = df_val.filter(items=['Relevant Texts', '3m'])
df_test_filtered = df_test.filter(items=['Relevant Texts', '3m'])
df_train_filtered.head()

,Relevant Texts,3m
2,"., perkinelmer inc.and agilent technologies, i...",0.0
10,agilent technologies: latest guidance increase...,0.0
11,competitive outlook... ... the key enterprises...,0.0
15,agilent technologies inc. (a) - get report tod...,0.0
16,company current portfolio ) buys invesco bulle...,1.0


In [23]:
df_train_filtered['3m'] = df_train_filtered['3m'].astype(int)
df_val_filtered['3m'] = df_val_filtered['3m'].astype(int)
df_test_filtered['3m'] = df_test_filtered['3m'].astype(int)

In [24]:
df_train_filtered = df_train_filtered.rename(columns={'Relevant Texts': 'text', '3m': 'label'})
df_val_filtered = df_val_filtered.rename(columns={'Relevant Texts': 'text', '3m': 'label'})
df_test_filtered = df_test_filtered.rename(columns={'Relevant Texts': 'text', '3m': 'label'})

In [25]:
df_train_filtered['text'].str.len().mean()

465.4702441678569

In [26]:
train = Dataset.from_pandas(df_train_filtered)
val = Dataset.from_pandas(df_val_filtered)
test = Dataset.from_pandas(df_test_filtered)

In [27]:
train_tf = train.to_tf_dataset(
    columns=['text', 'label'],
    batch_size=16
)
val_tf = val.to_tf_dataset(
    columns=['text', 'label'],
    batch_size=16
)
test_tf = test.to_tf_dataset(
    columns=['text', 'label'],
    batch_size=16
)

In [28]:
df_train_filtered

,text,label
2,"., perkinelmer inc.and agilent technologies, i...",0
10,agilent technologies: latest guidance increase...,0
11,competitive outlook... ... the key enterprises...,0
15,agilent technologies inc. (a) - get report tod...,0
16,company current portfolio ) buys invesco bulle...,1
...,...,...
57905,bear of the day: tal education (tal),1
57906,tal education group (a) un neutral by citigrou...,1
57910,the heavy-handed chinese government incited th...,1
57913,what happened... wednesday was not the best da...,1


In [29]:
train_tf.__len__()

<tf.Tensor: shape=(), dtype=int64, numpy=1838>

In [30]:
def make_bert_preprocess_model(sentence_features, seq_length=128):
  """Returns Model mapping string features to BERT inputs.

  Args:
    sentence_features: a list with the names of string-valued features.
    seq_length: an integer that defines the sequence length of BERT inputs.

  Returns:
    A Keras Model that can be called on a list or dict of string Tensors
    (with the order or names, resp., given by sentence_features) and
    returns a dict of tensors for input to BERT.
  """

  input_segments = [
      tf.keras.layers.Input(shape=(), dtype=tf.string, name=ft)
      for ft in sentence_features]

  # Tokenize the text to word pieces.
  bert_preprocess = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
  tokenizer = hub.KerasLayer(bert_preprocess.tokenize, name='tokenizer')
  segments = [tokenizer(s) for s in input_segments]

  # Optional: Trim segments in a smart way to fit seq_length.
  # Simple cases (like this example) can skip this step and let
  # the next step apply a default truncation to approximately equal lengths.
  truncated_segments = segments

  # Pack inputs. The details (start/end token ids, dict of output tensors)
  # are model-dependent, so this gets loaded from the SavedModel.
  packer = hub.KerasLayer(bert_preprocess.bert_pack_inputs,
                          arguments=dict(seq_length=seq_length),
                          name='packer')
  model_inputs = packer(truncated_segments)
  return tf.keras.Model(input_segments, model_inputs)

In [31]:
def build_classifier_model(num_classes):

  class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
      super(Classifier, self).__init__(name="prediction")
      self.encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)
      self.dropout = tf.keras.layers.Dropout(0.4)
      self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
      encoder_outputs = self.encoder(preprocessed_text)
      pooled_output = encoder_outputs["pooled_output"]
      x = self.dropout(pooled_output)
      x = self.dense(x)
      return x

  model = Classifier(num_classes)
  return model

In [32]:
sentence_features = ['text']
bert_preprocess_model = make_bert_preprocess_model(sentence_features)

In [33]:
train_tf = train_tf.map(lambda ex: (bert_preprocess_model(ex), ex['label']))

/usr/local/lib/python3.8/dist-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['label'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


In [34]:
val_tf = val_tf.map(lambda ex: (bert_preprocess_model(ex), ex['label']))

In [35]:
test_tf = test_tf.map(lambda ex: (bert_preprocess_model(ex), ex['label']))

In [36]:
epochs = 10
batch_size = 16
init_lr = 1e-5
train_data_size = 29406
validation_data_size = 4290

steps_per_epoch = train_data_size // batch_size
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = num_train_steps // 10
validation_steps = validation_data_size // batch_size


optimizer = optimization.create_optimizer(
      init_lr=init_lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      optimizer_type='adamw')
metrics = tf.keras.metrics.SparseCategoricalAccuracy(
        'accuracy', dtype=tf.float32)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [37]:
classifier_model = build_classifier_model(3)

In [38]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
weights_dir = '/drive/MyDrive/weights'
chkpt = ModelCheckpoint(filepath=weights_dir, monitor='loss', save_best_only=True, save_weights_only=True, mode='auto', period=1)
classifier_model.compile(optimizer=optimizer, loss=loss, metrics=[metrics])
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

history = classifier_model.fit(
    x=train_tf,
    validation_data=val_tf,
    shuffle=True,
    epochs=epochs,
    batch_size=16,
    callbacks=[chkpt, early_stop]
    )

Epoch 1/10
1838/1838 [==============================] - 919s 492ms/step - loss: 1.1556 - accuracy: 0.3715 - val_loss: 1.1452 - val_accuracy: 0.3832
Epoch 2/10
1838/1838 [==============================] - 909s 494ms/step - loss: 1.0872 - accuracy: 0.4054 - val_loss: 1.1013 - val_accuracy: 0.4200
Epoch 3/10
1838/1838 [==============================] - 947s 515ms/step - loss: 1.0644 - accuracy: 0.4384 - val_loss: 1.1194 - val_accuracy: 0.4396
Epoch 4/10
1838/1838 [==============================] - 907s 494ms/step - loss: 1.0058 - accuracy: 0.5039 - val_loss: 1.1642 - val_accuracy: 0.4448
Epoch 5/10
1838/1838 [==============================] - 909s 495ms/step - loss: 0.9238 - accuracy: 0.5677 - val_loss: 1.2579 - val_accuracy: 0.4336
Epoch 6/10
1838/1838 [==============================] - 908s 494ms/step - loss: 0.8402 - accuracy: 0.6211 - val_loss: 1.3879 - val_accuracy: 0.4140
Epoch 7/10
1838/1838 [==============================] - 908s 494ms/step - loss: 0.7695 - accuracy: 0.6643 - val_

In [41]:
def plot_history(metrics):
    """
    Plot the training history

    Args:
        metrics(str, list): Metric or a list of metrics to plot
    """
    history_df = pd.DataFrame.from_dict(history.history)
    sns.lineplot(data=history_df[metrics])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

In [42]:
plot_history(['loss', 'val_loss'])

NameError: ignored

In [39]:
classifier_model.evaluate(test_tf)

346/346 [==============================] - 59s 168ms/step - loss: 1.5384 - accuracy: 0.3812


[1.538361668586731, 0.38120245933532715]

In [54]:
# Saving the model

saved_model_path = './drive/MyDrive/weights/bert'

preprocess_inputs = bert_preprocess_model.inputs
bert_encoder_inputs = bert_preprocess_model(preprocess_inputs)
bert_outputs = classifier_model(bert_encoder_inputs)
model_for_export = tf.keras.Model(preprocess_inputs, bert_outputs)

print('Saving', saved_model_path)

# Save everything on the Colab host (even the variables from TPU memory)
save_options = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model_for_export.save(saved_model_path, include_optimizer=False,
                      options=save_options)

Saving ./drive/MyDrive/weights/bert


In [66]:
# Load the model back for testing
reloaded_model = tf.saved_model.load(saved_model_path)

In [68]:
with tf.device('/job:localhost'):
  test_sentence = ['Apple stock is falling']
  bert_result = reloaded_model(test_sentence)
  bert_result_class = tf.argmax(bert_result, axis=1)[0]
  print(bert_result)
  print(bert_result_class)

tf.Tensor([[ 0.49078482 -1.3280668   1.2229861 ]], shape=(1, 3), dtype=float32)
tf.Tensor(2, shape=(), dtype=int64)
